In [15]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product
from collections import OrderedDict
import time
import re

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops as so

ROOT = Path('../')
DATA_DIR = ROOT/'data'


In [42]:
# Car costs
# Use 30%/year diminishing value for cars taken from IRD depreciation calculator (https://interact1.ird.govt.nz/forms/depnrates/6f2b363212311e77353c7a1f373c6e2229472e70.continue;jsessionid=8C181312A1BA5BD98EE39396468C2E94)

def value(purchase_price, num_years, depreciation_rate=0.3):
    x = purchase_price
    n = num_years
    k = math.floor(n)
    kk = n - k
    r = depreciation_rate
    if n <= 0:
        d = 0
    else:
        d = sum([x*r**i for i in range(1, k + 1)]) + kk*x*r**(k + 1)
    return x - d

value(50679, 2)

30914.190000000002

# Compute Auckland transit fares


In [4]:
def get_journey_auckland(orig, dest, departure_time=None, max_walk=2000):
    """
    INPUT
    ------
    orig : list
        WGS84 longitude-latitude pair
    dest : list
        WGS84 longitude-latitude pair
    departure_time : string
        ISO 8601 datetime; e.g. '2017-06-01T07:30:00'
        
    OUTPUT
    ------
    dictionary
        Decoded JSON response of journey
    """
    url = 'https://api.at.govt.nz/v2/public-restricted/journeyplanner/silverRailIVU/plan'
    fromLoc ='{!s},{!s}'.format(orig[1], orig[0])
    toLoc ='{!s},{!s}'.format(dest[1], dest[0])
    if departure_time is None:
        departure_time = dt.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    date = departure_time + '+12:00'  # Add UTC offset
    params = {
        'from': 'from',
        'to': 'to',
        'fromLoc': fromLoc,
        'toLoc': toLoc,
        'timeMode': 'A',
        'date': date, 
        'modes': 'BUS,TRAIN,FERRY',
        'operators': '',
        'optimize': 'QUICK',
        'maxWalk': max_walk,
        'maxChanges': '-1',
        'routes': '',
        'subscription-key': '323741614c1c4b9083299adefe100aa6',
    }
    r = requests.get(url, params=params)
    
    # Raise an error if bad request
    r.raise_for_status()

    return r.json()         

def get_fare_auckland(journey):
    """
    """
    try:
        fare = journey['response']['itineraries'][0]['fareAdult']/100
    except:
        fare = None
    return fare

def collect_auckland_fares(rental_points, departure_time):
    """
    """
    # Get all pairs of points excluding equal points
    f = rental_points[['rental_area', 'geometry']].copy()
    rows = [[o[0], o[1].coords[0], d[0], d[1].coords[0]] for o, d in product(f.values, f.values) if o[0] != d[0]]
    f = pd.DataFrame(rows, columns=['orig_name', 'orig', 'dest_name', 'dest'])

    print('This will take about {:02f} minutes'.format(f.shape[0]*3.6/60))

    # Get journeys for each pair
    rows = []
    for __, row in f.iterrows():
        try:
            j = get_journey_auckland(row['orig'], row['dest'], departure_time=departure_time)
            fare = get_fare_auckland(j)
        except:
            fare = None
        rows.append([row['orig_name'], row['dest_name'], fare])

    g = pd.DataFrame(rows, columns=['orig_name', 'dest_name', 'fare'])
    return g


In [72]:
# Test some
orig = [174.7433853149414, -36.85517522550505]
dest = [174.79625701904297, -36.82550066651677]
%time j = get_journey_auckland(orig, dest)
j
get_fare_auckland(j)


CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 3.56 s


{'error': None,
 'response': {'engine': 'silverRailIVU',
  'itineraries': [{'duration': 4140000,
    'durationStr': 'an hour',
    'endTime': '2017-05-17T16:41',
    'endTimeStr': '4:41 pm',
    'fareAdult': 1000,
    'fareChild': 600,
    'fareError': None,
    'fareHopAdult': 645,
    'fareHopChild': 339,
    'fareHopTertiary': 498,
    'fareWarnings': {},
    'legs': [{'distance': '1162 metres',
      'distanceExact': 1162,
      'distanceStr': '1162 metres',
      'duration': 1020000,
      'durationStr': '17 min',
      'endLat': '-36.86023',
      'endLon': '174.75198',
      'endTime': '2017-05-17T15:49',
      'endTimeStr': '3:49 pm',
      'fareAdult': 0,
      'fareChild': 0,
      'fareHopAdult': 0,
      'fareHopChild': 0,
      'fareHopTertiary': 0,
      'from': 'from',
      'isFastLeg': False,
      'isFirst': True,
      'isLast': False,
      'legGeometry': {'length': 34,
       'points': 'zg}_Fcqpi`@G@e@}LGiAESlHkC^]NY`CsFfN_[pBlCKJ'},
      'messages': [],
      'mo

In [ ]:
path = DATA_DIR/'auckland'/'rental_points.geojson'
rp = gpd.read_file(str(path))
departure_time = '2017-06-01T07:30:00'
g = collect_auckland_fares(rp, departure_time)

path = DATA_DIR/'auckland'/'transit_costs.csv'
g.to_csv(str(path), index=False)
g


This will take about 570.360000 minutes


# Compute Wellington transit fares

In [32]:
# TODO: Changed to ordered params following http://stackoverflow.com/a/18490657
def get_journey_wellington(orig, dest, departure_time=None):
    """
    """
    url = 'https://www.metlink.org.nz/journeyplanner/JourneyPlannerForm'
    from_coords ='{!s},{!s}'.format(orig[1], orig[0])
    to_coords ='{!s},{!s}'.format(dest[1], dest[0])
    if departure_time is None:
        departure_time = dt.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    date = departure_time + '+12:00'  # Add UTC offset
    params = {
        'From': '12+Cambridge+Terrace,+Te+Aro,+Wellington+6011,+New+Zealand',
        'To': '220+The+Terrace,+Wellington,+6011,+New+Zealand',
        'Via': '',
        'When': 'LeaveAfter',
        'Date': '2017-05-18',
        'Time': '10:21+am',
        'MaxChanges': 5,
        'WalkingSpeed': 4,
        'MaxWalking': 2000,
        'Modes[Train]': 'Train',
        'Modes[Bus]': 'Bus',
        'Modes[Ferry]': 'Ferry',
        'Modes[Cable+Car]': 'Cable+Car',
        'ShowAdvanced': '',
        'FromCoords': from_coords,
        'ToCoords': to_coords,
        'ViaCoords': '',
        'action_doForm': 'Go',
    }
    r = requests.get(url, params=params)
    # Raise an error if bad request
    r.raise_for_status()
    return r.json()         

def get_fare_wellington(journey):
    """
    """
    pattern = 'Total adult fare </span><strong>&#36;(\d+\.\d\d)</strong>'
    m = re.search(pattern, journey)
    if m:
        fare = float(m.group(1))
    else:
        fare = None
    return fare

In [33]:
orig = (174.78368282318115, -41.294025458111015)
dest = (174.77248191833496, -41.28741516221087)
url = 'https://www.metlink.org.nz/journeyplanner/JourneyPlannerForm?From=12+Cambridge+Terrace,+Te+Aro,+Wellington+6011,+New+Zealand&To=220+The+Terrace,+Wellington,+6011,+New+Zealand&Via=&When=LeaveAfter&Date=2017-05-18&Time=10:21+am&MaxChanges=5&WalkingSpeed=4&MaxWalking=2000&Modes[Train]=Train&Modes[Bus]=Bus&Modes[Ferry]=Ferry&Modes[Cable+Car]=Cable+Car&ShowAdvanced=&FromCoords=-41.294025458111015,174.78368282318115&ToCoords=-41.28741516221087,174.77248191833496&ViaCoords=&action_doForm=Go'
#r = requests.get(url)
get_journey_wellington(orig, dest)

HTTPError: 404 Client Error: Not Found for url: https://www.metlink.org.nz/journeyplanner/JourneyPlannerForm?FromCoords=-41.294025458111015%2C174.78368282318115&Modes%5BBus%5D=Bus&When=LeaveAfter&ShowAdvanced=&Modes%5BCable%2BCar%5D=Cable%2BCar&ViaCoords=&MaxWalking=2000&To=220%2BThe%2BTerrace%2C%2BWellington%2C%2B6011%2C%2BNew%2BZealand&Time=10%3A21%2Bam&ToCoords=-41.28741516221087%2C174.77248191833496&Modes%5BFerry%5D=Ferry&Date=2017-05-18&Via=&WalkingSpeed=4&action_doForm=Go&MaxChanges=5&From=12%2BCambridge%2BTerrace%2C%2BTe%2BAro%2C%2BWellington%2B6011%2C%2BNew%2BZealand&Modes%5BTrain%5D=Train

In [28]:
get_fare_wellington(r.text)

2.0